In [ ]:
#imports
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

In [ ]:
# Connect to OpenAI, Anthropic
openai = OpenAI()
claude = anthropic.Anthropic()

Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models. Later we will be putting LLMs to better use!

What information is included in the API

Typically we'll pass to the API:

The name of the model that should be used
A system message that gives overall context for the role the LLM is playing
A user message that provides the actual prompt
There are other parameters that can be used, including temperature which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [ ]:
system_message ="You are an assistant that is great at telling jokes"
user_prompt="Tell a light-hearted joke for an audience of Data Scientiest"

In [ ]:
prompts = [
    { "role": "system", "content": system_message },
    { "role": "user", "content": user_prompt}
]

In [ ]:
# Open AI GPT-5 nano
completion = openai.chat.completions.create(
    model="gpt-5-nano",
    messages=prompts
)
print(completion.choices[0].message.content)

In [ ]:
# GPT-4o-mini
# Temperature setting controls creativity
completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

In [ ]:
# GPT-4o
completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

model:
Specifies which Anthropic model to use (e.g., 'claude-3-7-sonnet-latest'). This determines the capabilities and behavior of the AI.

max_tokens:
The maximum number of tokens (words and punctuation) the model can generate in its response. Limits the length of the output.

temperature:
Controls the randomness/creativity of the output.

Lower values (e.g., 0.2) make the output more focused and deterministic.
Higher values (e.g., 0.8) make the output more random and creative.
system:
A string that sets the overall behavior, persona, or instructions for the assistant (the “system prompt”).

messages:
A list of message objects (dictionaries) representing the conversation history.
Each message has a "role" (like "user" or "assistant") and "content" (the actual text).
In this case, it’s a single user message with your joke prompt.

In [ ]:
# Claude claude-3.7
# Also adding max_tokens

message = claude.messages.create(
    model='claude-3-7-sonnet-latest',
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[{"role": "user", "content": user_prompt}],
)

print(message.content[0].text)

In [ ]:
# Claude 3.7 Sonnet again
# Now let's add in streaming back results
# If the streaming looks strange, then please see the note below this cell!

result = claude.messages.stream(
    model='claude-3-7-sonnet-latest',
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[{
        "role": "user", "content": user_prompt
    }]
)

with result as stream:
    for text in stream.text_stream:
        print(text, end="", flush=True)

In [ ]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very arargumentative; \
              You disagree with anything in the conversation and you challenge everything, in a snarky way."


claude_system = "You are a very polite, courteous chatbot. You try to agree with \
                everything the other person says, or find common ground. If the other person is argumentative, \
                you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]


In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
        completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
        
    return completion.choices[0].message.content

In [ ]:
call_gpt()

In [ ]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [ ]:
call_claude()

In [ ]:
call_gpt()

In [ ]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")


for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)